In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from Yelp_scraper import yelp_search_page
from Yelp_scraper import yelp_store_page
from tqdm import tqdm
import time
import csv
import json

def init_driver(driver_path):
    '''
    Get Selenium driver
    Parameters: None
    Return: Selenium webdriver element
    '''
    opts = Options()

    opts.add_argument("user_agent = Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.20 Safari/537.36")
    opts.add_argument("--disable-notifications")
    opts.add_argument('--headless')
    opts.add_argument("--window-size=720,1080")
    opts.add_argument("--lang=en")
    opts.add_argument('--no-sandbox')
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument('disable_infobars')
    drive = webdriver.Chrome(driver_path, options=opts)
        
    return drive

def read_csv(filepath):
    csv_data = []
    try:
        with open(filepath,'r',encoding='utf-8') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            for index,row in enumerate(csv_reader):
                csv_data.append(row)
    except Exception as e:
        print(e)
        with open(filepath,'r',encoding='utf-16') as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            for index,row in enumerate(csv_reader):
                csv_data.append(row[0].split('\t'))
    return csv_data

if __name__ == '__main__':
    drive_path = 'C:/Users/ljw15/chromedrive87/chromedriver_win32/chromedriver'
    driver = init_drive(drive_path)

    search_page = yelp_search_page(driver)
    store_page = yelp_store_page(driver)
    
    csv_file = 'E:/H/Forwork/TRDRP/yelp_shop/vidya_study/LR-CA-Statewide.csv'
    csv_data = read_csv(csv_file)
    
    #target_file = open('/data2/TRDRP/Yelp_data/vidya_project.json', 'w', encoding = 'utf-16')

    for index,d in enumerate(csv_data[1:]):
        LR_CA_id = d[0]
        store_name = d[2]
        store_address = d[3] + ' , ' + d[4] + ' , ' + d[5]
        street_num = d[3].split(' ')[0]
        search_page.search(store_name,store_address)
        if search_page.has_no_result():  #the search page return no result
            search_page.search('',store_address)  #the store name may mislead the result
            
        time.sleep(1)
        page_stores = search_page._get_page_stores()
        ########find matched address
        matched = False
        
        while matched == False:
            for store in page_stores:
                address = store['address']
                if address == False:
                    continue
                link = store['link']
                yelp_street_num = address.split(' ')[0]
                if yelp_street_num == street_num:
                    matched = True
                    store_page.go_to(link)
                    time.sleep(1)
                    store_info = store_page.get_store_info(link)
                    store_info['LR_CA_id'] = LR_CA_id
                    #json.dump(store_info, target_file)
                    #target_file.write('\n')
                    print(str(index),str(LR_CA_id),store_info)
                    break
            
            if search_page._has_next_page() and matched == False:
                search_page._move_to_next_page()
                time.sleep(1)
                page_stores = search_page._get_page_stores()
            else:   ####reach end of search page and has no result
                break

        if matched == False:
            print(str(index),str(LR_CA_id),"No result!!!!")

    target_file.close()


0 91204989 {'link': 'https://www.yelp.com/biz/bobs-market-santa-monica?osq=BOB%27S+FOOD+MART', 'name': 'Bobs Market', 'rate': '4 star rating', 'num_reviewer': '214 reviews', 'price': '$$', 'attribute': ['Grocery'], 'is_claimed': True, 'coordinate': {'latitude': '34.012208', 'longitude': '-118.466907'}, 'address': '1650 Ocean Park Blvd,Santa Monica, CA 90405', 'LR_CA_id': '91204989'}
1 91204990 No result!!!!
2 91204998 {'link': 'https://www.yelp.com/biz/winchester-fuels-76-temecula?osq=PROMENADE+76', 'name': 'Winchester Fuels 76', 'rate': '3.5 star rating', 'num_reviewer': '18 reviews', 'price': '$', 'attribute': ['Convenience Stores', 'Gas Stations'], 'is_claimed': False, 'coordinate': {'latitude': '33.526093', 'longitude': '-117.157150'}, 'address': '40720 Winchester Rd,Temecula, CA 92591', 'LR_CA_id': '91204998'}
3 91205003 No result!!!!



KeyboardInterrupt

